## Modelling

### Hierarchical Models
We have tried to modularize majority of the model training into functions that can be reused for many different types of models and for different hierarchy levels. Since the data is so vast it is harder for models to converge especially if they have items that show very different behavior, so we aim to create models similiar timeseries together. We achieve this by creating subsets of data using on a single particular store/category/department while modeling. In this scenario we have N different models for the N different levels in the categorical column. In some cases we have also done this for nested hierarchies such as creating models per store per category.

In [ ]:
import warnings
import pickle
import random
import gc
from itertools import product

import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb

SEED = 42
warnings.filterwarnings("ignore")
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Path constants
DATA_DIR = "../../processed_data/"

BASE = DATA_DIR+"base_sales.pkl"
PRICE = DATA_DIR+"price_features.pkl"
CALENDAR = DATA_DIR+"calendar_feats.pkl"
TS_FEATS = DATA_DIR+"time_based_feats.pkl"
CAT_ENC = DATA_DIR+"cat_encodings.pkl"

BASE_DIR = "../../"
MODEL_NAME = "lgb"
TEST_DATA_DIR = BASE_DIR + f"models/{MODEL_NAME}/"
MODELS_DIR = BASE_DIR + f"models/{MODEL_NAME}/"
MODEL_FILE_PREFIX = f"{MODEL_NAME}_model_"

TARGET_COL = "units_sold"
PRED_LENGTH = 28

TRAIN_END = 1941

# Change this list to lists we want to iterate through (unique values in the hierarchical columns)
CATEGORIES = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
STATES = ["CA","TX","WI"]
STORES = ["CA_1", "CA_2", "CA_3", "CA_4", "TX_1", "TX_2", "TX_3", "WI_1", "WI_2", "WI_3"]
DEPTS = ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1' ,'FOODS_2','FOODS_3']

HIERARCHY_COL = ["store_id","dept_id"]

# Include hierarchy cols in the drop features (as they will be constant for each model iteration)
# Remove the hierarchy cols not being used. 
# Example: If you're modelling by category, don't include store/depratment id in the drop features
drop_features = [
                "item_id",
                "dept_id",
                "cat_id",
                "store_id",
                "state_id",
                "date",
                "wm_yr_wk",
                "weekday"]
METRIC = "rmse"

### Data Preparation
We join all the features created from the various datasets and take a subset based on the hierarchical column values. We then split the data where day 1 to 1913 is the training data, 1914 to 1941 is the validation data and 1942 to 1969 is the test data currently with target column empty (eventually populated with the predictions). We save the test file which is used to generate predictions after the model is trained.

In [ ]:
def get_data_by_subset(col_names, col_ids):
    """
    Read and concat all the created features for a single value of a category/hierearchy
    `col_names` should be a list of all the column names we want to subset with.
    `col_ids` should be specific values we want the subset dataframe to have.
    """
    ## Works for multiple column subset
    df = pd.read_pickle(BASE)
    for index, column in enumerate(col_names):
        df = df[(df[column]==col_ids[index])]

    price_feats = pd.read_pickle(PRICE).loc[df.index,:].iloc[:,2:]
    calendar_feats = pd.read_pickle(CALENDAR).loc[df.index,:].iloc[:,2:]
    enc_feats = pd.read_pickle(CAT_ENC).loc[df.index,:].iloc[:,2:]
    lag_feats = pd.read_pickle(TS_FEATS).loc[df.index,:].iloc[:,2:]
    
    # Adding mean and std features 
    # (but we do not include the mean and std for the hierarchical columns)
    remove_col_prefix = col_names[:]+["_".join(col_names)]
    excluded_encoding = [col+suffix for col in remove_col_prefix for suffix in ["_mean","_std"]]
    encoding_features = [i for i in enc_feats.columns if i not in excluded_encoding]
    enc_feats = enc_feats.loc[:, encoding_features]
    df = pd.concat([df,
                    price_feats,
                    calendar_feats,
                    enc_feats,
                    lag_feats],
                    axis=1)
    df["d"] = df["d"].str.replace("d_","").astype(np.int16)
    
    del price_feats, calendar_feats, enc_feats, lag_feats
    # Subsetting columns with only the required features
    features = [column for column in df.columns if column not in drop_features]
    df = df[features]
    df = df.reset_index(drop=True)
    
    # Id is needed in the test data but not training
    features.remove(TARGET_COL)
    features.remove("id")
    return df, features

In [ ]:
def split_data(df: pd.DataFrame, col_ids: list):
    train_indices = df["d"]<=TRAIN_END
    valid_indices = train_indices & (df["d"]>(TRAIN_END-PRED_LENGTH))
    preds_indices = (df["d"]>(TRAIN_END-100)) & (df["d"] <= TRAIN_END+PRED_LENGTH)

    train_features = df[train_indices][MODEL_FEATURES]
    train_labels= df[train_indices][TARGET_COL]

    valid_features = df[valid_indices][MODEL_FEATURES]
    valid_labels = df[valid_indices][TARGET_COL]

    test = df[preds_indices].reset_index(drop=True)
    del df

    substitute = test[TARGET_COL].values
    substitute[(test["d"] > TRAIN_END)] = np.nan
    test[TARGET_COL] = substitute

    # saving test data
    file_uid = "_".join(col_ids)
    test.to_pickle(MODELS_DIR+"test_data_"+file_uid+".pkl")
    del test, substitute
    gc.collect()
    return valid_features,valid_labels,train_features,train_labels

### Tuning
For both XGBoost and LightGBM we create a hyperparameter space in which we can search through a huge number of configurations. We explore all the major paramters with wide ranges i.e. big search space. These wide ranges for parameters means that we can search through alot of options and configurations giving us a good chance to find the best set of hyperparameters. We use bayesian optimization to save time (as grid search would be very time taking for such huge search space and random search has no guarentee that our solution will be close to optimal).

### XGBoost
__Normalization__: Since xGBoost is an ensemble model of decision trees we do not need to normalize our data.

__Hyperparameter Tuning__: In XGBoost we explore all the major paramters with wide ranges. These wide ranges for parameters means that we can search through alot of options and configurations giving us a good chance to find the best set of hyperparameters.

- `n_estimators`: For Number of estimators we search from 500 to 3000 decision trees in the model.
- `learning_rate`: We vary our learning rate from 0.01 to 0.1
- `subsample`: The subsample (fraction) of the data taken varies from 50% of the data to 100%
- `gamma`: From 0 to 0.5
- `min_child_weight`: The minimum child weight for each leaf node varies from 1 to 6
- `max_depth`: The maximum depth of the tree varies from 5 to 50
- `lambda`: The L2 regularization parameter varies from 0.25 to 1
- `alpha`: The L1 regularization parameter varies from 0.25 to 1

We run 50 different configurations of the model.

### LightGBM
__Normalization__: Since `lightGBM` is an ensemble model of decision trees we do not need to normalize our data.

__Hyperparameter Tuning__: In `lightGBM` we explore the following hyperparamters:
- `num_leaves`: We vary the number of leaves in each tree from 5 to 500.
- `learning_rate`: The learning rate of the `lightGBM` model varies from 0.001 to 0.2
- `max_depth`: We vary the depth of the decision tree from 5 to 50
- `n_estimators`: The number of estimator trees varies from 500 to 3000
- `min_child_samples`: We vary the minimum child samples required in each tree from 0.001 to 0.2
- `reg_lambda`: The L2 regularization parameter varies from 0.1 to 1
- `reg_alpha`: The L2 regularization parameter varies from 0.1 to 1
- `colsample_bytree`: We vary the number of features taken in a tree from 0.5 (50% of all the features) to 1.

### Running Tuning and Final Training
We run 50 different configurations for each hierarchical level for both of these model on a smaller sample of the dataset containing around 10% of the total item ids (349). We use a custom class `ModelExecutor` which abstracts all this and generates the best param space corresponding to the lowest RMSE. This hyperparameter configuration is then used to train the final model with the whole dataset. This process is repeated as we iterate through the various hierarchical levels (which can be one or multiple).

In [ ]:
# Tuning constants
OBJECTIVE = "min"
TASK = "regression"
NUM_TRIALS = 50
SAMPLE_SIZE = 349

# Light GBM Hyper paramter space
lgb_hp_space = {"num_leaves": (2, 50, 2),
            "max_depth": (2, 50, 2),
            "learning_rate": (0.01, 0.1, 0.01),
            "n_estimators": (20, 500, 20),
            "min_child_samples": (5, 100, 5), 
            "reg_lambda": (0.1, 1, 0.1),
            "reg_alpha": (0.1, 1, 0.1),
            "boosting": ["gbdt","dart"],
            "max_bin": (50, 200, 50),
            "tweedie_variance_power": (1.0, 1.5, 0.1),
            "subsample": (0.4, 0.9, 0.1),
            "feature_fraction": (0.5, 1, 0.1),
            "subsample_freq": 1,
            "random_state": 42,
            "objective": "tweedie",
            "metric": "rmse"}

xgb_hp_space = {
    "n_estimators": (500,3000,500),
    "learning_rate": (0.01,0.1,0.01),
    "subsample": (0.5, 1, 0.1),
    "gamma": (0,0.5,0.1),
    "min_child_weight": (1,6,2),
    "max_depth": (5,50,5),
    "lambda": (0.25,1,0.25),
    "alpha": (0.25,1,0.25),
    "colsample_bytree": (0.5, 1, 0.1),
    "objective": 'reg:squarederror',
    "random_state": 42,
    "eval_metric": METRIC,
}

In [ ]:
# TRYING TUNING
from basic_ml import ModelExecutor

def tune_model(valid_features,
                valid_labels,
                train_features,
                train_labels,
                col_ids,
                model_class,
                model_name,
                tune_idx,
                model_hp_space: dict={},
                model_fit_params: dict={}):
    
    # Tuning models for the sample ids
    model_tuner = ModelExecutor(model_class=model_class,
                            x_train=train_features[tune_idx],
                            y_train=train_labels[tune_idx],
                            param_grid=model_hp_space,
                            fit_params=model_fit_params,
                            objective=OBJECTIVE,
                            normalize=False,
                            cv=False,
                            task=TASK,
                            internal_val=True,
                            x_val=valid_features[tune_idx],
                            y_val=valid_labels[tune_idx],
                            final_train_flag=False)
    model_tuner.execute(max_evals=NUM_TRIALS)
    # performing final model training on the whole dataset
    final_model = model_tuner.base_model.set_params(**model_tuner.best_params)
    final_model.fit(pd.concat([train_features,valid_features]),
                        pd.concat([train_labels,valid_labels]))
    # Saving model and clean up
    del valid_features, valid_labels, train_features, train_labels, model_tuner
    save_model(col_ids, model_name, final_model)
    gc.collect()

def save_model(col_ids, model_name, model):
    """
    Save model using the column id and model name.
    Directory should be created prior to running the function.
    """
    print("model trained for: ", col_ids)
    file_uid = "_".join(col_ids)
    model_path = MODELS_DIR+model_name+"_"+file_uid+".pkl"
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    del model

In [ ]:
# Running tuning for all stores & categories
for col_ids_iter in list(product(STORES, DEPTS)):
    col_ids = list(col_ids_iter) # Converting tuple to list
    
    sub_df, MODEL_FEATURES = get_data_by_subset(col_names=HIERARCHY_COL,
                                    col_ids=col_ids)
    random_idx = np.random.randint(0, SAMPLE_SIZE, SAMPLE_SIZE)
    tune_idx = sub_df['id'].isin(sub_df['id'].unique()[random_idx])

    valid_features,valid_labels,train_features,train_labels = split_data(sub_df, col_ids)
    del sub_df

    tune_model(valid_features,
                valid_labels,
                train_features,
                train_labels,
                col_ids=col_ids,
                model_class=lgb.LGBMRegressor,
                model_name=f"{MODEL_NAME}_model",
                model_hp_space=lgb_hp_space,
                tune_idx=tune_idx)